<a href="https://colab.research.google.com/github/emirozyesil/DSA-210-term-project/blob/main/notebooks/01_data_cleaning_enrichment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import zipfile
import os


zip_path = "/content/TeamStatistics.csv.zip"


if "TeamStatistics.csv" not in os.listdir("/content"):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content")

print("Files in content folder:", os.listdir("/content"))
team_stats_path = "/content/TeamStatistics.csv"
nba_path = "/content/nba.csv"

team = pd.read_csv(team_stats_path)
nba = pd.read_csv(nba_path)

team.head(), nba.head()
team.columns
nba.columns

team["gameDateTimeEst"] = pd.to_datetime(team["gameDateTimeEst"], errors="coerce")


team["teamName"] = team["teamName"].str.upper().str.strip()
team["opponentTeamName"] = team["opponentTeamName"].str.upper().str.strip()

team[["gameId", "gameDateTimeEst", "teamName", "opponentTeamName"]].head()

team["twoPointersAttempted"] = (
    team["fieldGoalsAttempted"] - team["threePointersAttempted"]
)


denom = team["twoPointersAttempted"] + team["threePointersAttempted"]


team["threePA_ratio"] = np.where(
    denom > 0,
    team["threePointersAttempted"] / denom,
    np.nan
)

team[["fieldGoalsAttempted", "threePointersAttempted", "twoPointersAttempted", "threePA_ratio"]].head()


q3_self = team[["gameId", "teamId", "q3Points"]]


q3_opp = q3_self.rename(columns={
    "teamId": "opponentTeamId",
    "q3Points": "opponentQ3Points"
})


team = team.merge(
    q3_opp,
    on=["gameId", "opponentTeamId"],
    how="left"
)


team["Q3_diff"] = team["q3Points"] - team["opponentQ3Points"]

team[["gameId", "teamName", "q3Points", "opponentQ3Points", "Q3_diff"]].head()

foul_self = team[["gameId", "teamId", "foulsPersonal"]]


foul_opp = foul_self.rename(columns={
    "teamId": "opponentTeamId",
    "foulsPersonal": "opponentFoulsPersonal"
})


team = team.merge(
    foul_opp,
    on=["gameId", "opponentTeamId"],
    how="left"
)


team["Foul_diff"] = team["foulsPersonal"] - team["opponentFoulsPersonal"]

team[["gameId", "teamName", "foulsPersonal", "opponentFoulsPersonal", "Foul_diff"]].head()

team["final_margin"] = team["teamScore"] - team["opponentScore"]


team["close_game"] = (team["final_margin"].abs() <= 5).astype(int)

team[["gameId", "teamName", "teamScore", "opponentScore", "final_margin", "close_game"]].head()

cols = [
    "gameId",
    "gameDateTimeEst",
    "teamName",
    "opponentTeamName",
    "home",
    "win",
    "teamScore",
    "opponentScore",
    "q1Points", "q2Points", "q3Points", "q4Points",
    "Q3_diff",
    "foulsPersonal", "opponentFoulsPersonal", "Foul_diff",
    "fieldGoalsAttempted", "threePointersAttempted", "twoPointersAttempted",
    "threePA_ratio",
    "final_margin", "close_game"
]

df_enriched = team[cols].copy()

df_enriched.head()
output_path = "/content/nba_games_enriched.csv"
df_enriched.to_csv(output_path, index=False)

print("Dataset saved to:", output_path)


Files in content folder: ['.config', 'TeamStatistics.csv', '__MACOSX', 'nba.csv', 'TeamStatistics.csv.zip', 'sample_data']
Dataset saved to: /content/nba_games_enriched.csv
